1. [算法原理](#算法原理)
2. [损失函数](#损失函数)
3. [分裂结点算法](#分裂结点算法)
4. [正则化](#正则化)
5. [对缺失值处理](#对缺失值处理)
6. [优缺点](#优缺点)
7. [应用场景](#应用场景)
8. [XGB参数](#XGB参数)

# 算法原理

xgb也是加法模型,  
$$\hat{y}_i=\phi(x_i)=\sum_{k=1}^{K}f_k(x_i), f_k\in \mathcal F \\  其中\mathcal F=\{f(x)=\omega_{q(x)}\}(q:\mathcal R^m \rightarrow T, \omega\in \mathcal R^T) 为CART回归树$$

优化目标为
$$\mathcal L(\phi)=\sum_il(\hat{y}_i, y_i) + \sum_k\Omega(f_k), \\ \Omega(f)=\gamma T + \frac{1}{2}\lambda \|\omega\|^2$$其中$l$为衡量误差的可微凸损失函数,$\Omega$为模型复杂度的正则项
# 损失函数
GBDT用一阶梯度来拟合损失, xgb则是用的二阶梯度来拟合损失,

具体的
$$\mathcal L^{(t)} = \sum_{i=1}^{n}l(y_i, \hat{y}_i^{(t-1)}+f_t(x_i)) + \Omega(f_t)$$

二阶近似
$$\mathcal L^{(t)} \simeq \sum_{i=1}^{n}[l(y_i, \hat{y}^{(t-1)} + g_if_t(x_i) + \frac{1}{2}h_if_t^2(x_i)] + \Omega(f_t) \\ 其中g_i=\partial_{\hat{y}^{(t-1)}}l(y_i, \hat{y}^{(t-1)}), h_i=\partial_{\hat{y}^{(t-1)}}^{2}l(y_i,\hat{y}^{(t-1)})为损失的一阶二阶梯度$$

去掉常量后的简化目标
$$\tilde{\mathcal L}^{(t)} = \sum_{i=1}^{n}[g_if_t(x_i)+\frac{1}{2}h_if_t^2(x_i)]+\Omega(f_t) $$

将正则项展开
$$\tilde{\mathcal L}^{(t)}=\sum_{i=1}^{n}[g_if_t(x_i)+\frac{1}{2}h_if_t^2(x_i)]+\gamma T+\frac{1}{2}\lambda\sum_{j=1}^{T}\omega_j^2$$

对同一子树$I_j=\{i|q(x_i)=j\}$,将其样本子叶j归结到一起
$$\tilde{\mathcal L}^{(t)} = \sum_{j=1}^{T}[(\sum_{i\in I_j}g_i)\omega_j+\frac{1}{2}(\sum_{i\in I_j}h_i+\lambda)\omega_j^2]+\gamma T$$

对一固定的树结构q(x)

仅通过子叶j即可计算出其最优权重
$$\omega_j^*=-\frac{\sum_{i\in I_j}g_i}{\sum_{i\in I_j}h_j + \lambda}$$

最优值
$$\tilde{\mathcal L}^{(t)}(q) = -\frac{1}{2}\sum_{j=1}^{T}\frac{(\sum_{i\in I_j}g_i)^2}{\sum_{i\in I_j}h_i +\lambda} + \gamma T$$

所以我们会用这最优值来衡量q(x)的好坏, 类似与决策树的不纯度计算

通常是不可能去搜索所有q(x)的, xgb使用的是一种贪心算法, 从叶结点开始迭代,不断地给树添加分支




# 分裂结点算法
假定划分的$右结点为I_R, 左结点为I_L, I=I_L+I_R$,则划分的损失的增益可通过下式计算$$\mathcal L_{split} = \frac{1}{2}\left[\frac{(\sum_{i\in I_L}g_i)^2}{\sum_{i\in I_L}h_i + \lambda} + \frac{(\sum_{i\in I_R}g_i)^2}{\sum_{i\in I_R}h_i + \lambda} - \frac{(\sum_{i\in I}g_i)^2}{\sum_{i\in I}h_i + \lambda}\right] - \gamma$$
选择增益最大的特征来划分结点

> Exact Greedy Algorithm

根据损失增益, 枚举所有特征, 择其最优
> Weighted Quantile Sketch

根据特征分布的分位数进行划分

对集合$\mathcal D_k = \{(x_{1k}, h_1), (x_{2k}, h_2),...,(x_{nk}, h_n)\},$其中$x_{ik}$代表第k个特征的值, $h_k$指的是样本的二阶梯度值. 定义rank函数
$$r_k: \mathcal R \rightarrow [0,+\infty) \\ r_k(z)=\frac{1}{\sum_{(x,h)\in \mathcal D_k}h}\sum_{(x,h)\in \mathcal D_k, x<z}h$$
来表示第k个特征小于z的比例

找到划分点{$s_{k1},s_{k2},...,s_{kl}$}, 且满足
$$|r_k(s_{k,j})-r_k(s_{k,j+1}|<\epsilon,\space\space s_{k1}=\min_ix_{il}, s_{kl}=\max_ix_{ik}$$
其中$\epsilon是近似因子, 表示这里大概有1/\epsilon$个划分点.

这里的h看有被看作是权重, 将目标函数改写为$$\sum_{i=1}^{n}\frac{1}{2}h_i(f_t(x_i)-g_i/h_i)^2 + \Omega(f_t)+constant$$

# 对缺失值处理
xgb的每个树节点都有一个默认的划分方向, 当出现缺失值的时候, 会按照默认的划分方向进行划分

默认方向通过数据学习出来, 致使增益最大的方向, 每一个分支在默认的划分方向上都有两种划分选择

# 正则化
除了目标函数中的$\Omega(f_k)$, xgb还采用了两种额外的正则方法
- Shrinkage
Shrinkage给新的子树权重添加了一个收缩因子$\eta$, 类似SGD优化中学习率, 减少了树与树之间的影响, 给接下来生成的子树预留了学习空间,
- 特征(列)下采样
类似与随机森林中的采样, 大大减少了计算量, 同时能加速xgb的并行算法

# 优缺点
> 优点

- 1. 端到端的提升树系统
- 2. 用基于分位数的分裂方法进行高效计算, 有理论证明,
- 3. 引入了一种sparsity-aware算法来实现并行化
- 4. 提出了一种高效的cache-aware的块结构来进行树模型的out-of-core学习
- 5. 支持自定义loss function

> 缺点

- xgBoosting采用预排序，在迭代之前，对结点的特征做预排序，遍历选择最优分割点，数据量大时，贪心法耗时，LightGBM方法采用histogram算法，占用的内存低，数据分割的复杂度更低
- xgBoosting采用level-wise生成决策树，同时分裂同一层的叶子，从而进行多线程优化，不容易过拟合，但很多叶子节点的分裂增益较低，没必要进行跟进一步的分裂，这就带来了不必要的开销；LightGBM采用深度优化，leaf-wise生长策略，每次从当前叶子中选择增益最大的结点进行分裂，循环迭代，但会生长出更深的决策树，产生过拟合，因此引入了一个阈值进行限制，防止过拟合.

# 应用场景

基于树模型的XGBoost则能很好地处理表格数据，同时还拥有一些深度神经网络所没有的特性（如：模型的可解释性、输入数据的不变性、更易于调参等）。

# XGB参数
|参数|用途
|:-:|:-:
|booster|gbtree 树模型做为基分类器（默认）,gbliner 线性模型做为基分类器
|njobs|多线程, -1时使用全部CPU进行并行运算
|learning_rate| Boosting learning rate (xgb's "eta")
|gamma|划分子节点的最低损失增益
|subsample| 下采样比率
|colsample_bytree|列采样比率
|colsample_bylevel|每棵树每次节点分裂的时候列采样的比例
|reg_alpha|xgb's alpha
|reg_lambda|xgb's lambda
|max_delta_step|限制了每棵树权重改变的最大步长，如果这个参数的值为0,则意味着没有约束
|objective|目标函数,linear(默认)
